In [37]:
!pip install -q langchain_community
!pip install -q pypdf
!pip install -q -U langchain-text-splitters
!pip install -q -U langchain
!pip install -q -U bitsandbytes
!pip install -q -U accelerate
!pip install -q -U chromadb
!pip install -q -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 1.6 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from huggingface_hub import notebook_login

notebook_login()

In [38]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from transformers import BitsAndBytesConfig
from langchain.prompts import PromptTemplate
from langchain_community.embeddings import LlamafileEmbeddings
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

model_name = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=nf4_config)

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [42]:
from langchain_community.document_loaders import PyPDFLoader

pdf = PyPDFLoader("/content/drive/MyDrive/colab_notebooks/Computer_vision.pdf")
pages = pdf.load_and_split()


In [72]:
template = """
<s>[INST]<<SYS>>Answer the question only based on details in the context string below but dont mention about context.
If the answer is not in the context, reply "I dont know".
Context: {context}
<</SYS>>
Query: {query} [/INST]
"""

prompt = PromptTemplate.from_template(template)
fprompt = prompt.format(context="My give name is venkata ratnam, my friends call me Venkat", query="what is my name ?")

fprompt

'\n<s>[INST]<<SYS>>Answer the question only based on details in the context string below but dont mention about context. \nIf the answer is not in the context, reply "I dont know".\nContext: My give name is venkata ratnam, my friends call me Venkat\n<</SYS>>\nQuery: what is my name ? [/INST]\n'

In [9]:
inputs = tokenizer(fprompt, return_tensors="pt")

# Generate
generate_ids = model.generate(inputs.input_ids, max_length=150)
ans = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

print (ans)


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1477: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(



 [INST]<<SYS>>Answer the question only from the context below, if the answer is not in the context, reply "i dont know".
Context: My give name is venkata ratnam, my friends call me Venkat
<</SYS>>
Query: what is my name ? [/INST]
Hello Venkata Ratnam! Your friends call you Venkat, so your name is Venkat.


In [10]:
fprompt = prompt.format(context="My give name is venkata ratnam, my friends call me Venkat", query="what is the size of pacific ocean ?")

print (fprompt)
inputs = tokenizer(fprompt, return_tensors="pt")

generate_ids = model.generate(inputs.input_ids, max_length=150)
tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]



<s>[INST]<<SYS>>Answer the question only from the context below, if the answer is not in the context, reply "i dont know".
Context: My give name is venkata ratnam, my friends call me Venkat
<</SYS>>
Query: what is the size of pacific ocean ? [/INST]



/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1477: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


'\n [INST]<<SYS>>Answer the question only from the context below, if the answer is not in the context, reply "i dont know".\nContext: My give name is venkata ratnam, my friends call me Venkat\n<</SYS>>\nQuery: what is the size of pacific ocean ? [/INST]\nI\'m just an AI, I don\'t have access to real-time information or data, but I can tell you the approximate size of the Pacific Ocean based on available data.\n\nThe Pacific Ocean is the largest of the Earth\'s five oceans, covering an area of approximately 155.6 million square kilometers'

In [40]:
# Define the Text Splitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from chromadb.utils import embedding_functions
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150
)

splits = text_splitter.split_documents(pages)
tokenizer.pad_token = tokenizer.eos_token
persist_directory = './docs/chroma/'
default_ef = embedding_functions.DefaultEmbeddingFunction()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
vectordb = Chroma.from_documents(
    documents=text_splitter.split_documents(pages),
    embedding=embeddings,
    persist_directory=persist_directory
)
print(vectordb._collection.count())

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

75


In [49]:
question = "what are different fields of computer vision ?"

# Similarity search with k = 5
docs = vectordb.similarity_search(question,k=5)

context = ""

for d in docs:
  context += d.page_content

fprompt = prompt.format(context=context, query=question)

inputs = tokenizer(fprompt, return_tensors="pt")

generate_ids = model.generate(inputs.input_ids, max_length=2000)
ans = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

print(ans[len(fprompt)-2:])

Based on the context provided, the different fields of computer vision are:

1. Image processing: This field deals with the processing and analysis of digital images.
2. Machine vision: This field involves the use of computer vision techniques to automate tasks that the human visual system can do.
3. Fashion eCommerce: This field uses computer vision techniques to analyze and understand fashion products, such as clothing and accessories, in order to improve e-commerce experiences.
4. Inventory management: This field uses computer vision to manage and track inventory, such as products and goods, in a variety of industries.
5. Patent search: This field uses computer vision to search and analyze patents related to specific technologies or inventions.
6. Furniture: This field uses computer vision to analyze and understand furniture, such as furniture design and layout, in order to improve furniture design and manufacturing.
7. Beauty industry: This field uses computer vision to analyze and

In [55]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [56]:
question = "Am i subscribed to MTV channel ?"

# Similarity search with k = 5
docs = vectordb.similarity_search(question,k=5)

context = """
Account name: John Doe
Subsciptions: Premium,
Premium subscription channels: MTV, CNN, Canal+, ESPN, Star sports
"""

fprompt = prompt.format(context=context, query=question)

inputs = tokenizer(fprompt, return_tensors="pt").to(device)

generate_ids = model.generate(inputs.input_ids, max_length=2000)
ans = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

print(ans[len(fprompt)-2:])

Yes, you are subscribed to the MTV channel.


In [76]:
question = "Am i subscribed to BBC ?"

# Similarity search with k = 5
docs = vectordb.similarity_search(question,k=5)

context = """
Account name: John Doe
Subsciptions: Premium product
User is subscribed to channels MTV, CNN, Canal+, ESPN, Star sports only and other channels are not suscribed.
"""

fprompt = prompt.format(context=context, query=question)

inputs = tokenizer(fprompt, return_tensors="pt").to(device)

generate_ids = model.generate(inputs.input_ids, max_length=2000)
ans = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

print(ans[len(fprompt)-2:])

No, you are not subscribed to BBC. According to the information provided in the context, you are only subscribed to the following channels:

* MTV
* CNN
* Canal+
* ESPN
* Star sports

There is no mention of BBC in the context, so I cannot confirm whether you are subscribed to it or not.
